<a href="https://colab.research.google.com/github/tranvantung2002/CSDLNC/blob/main/viXTTS_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔥🔥🔥**viXTTS Demo**🗣️🗣️🗣️

Demo này giúp bạn chạy viXTTS miễn phí trên Google Colab!
Xem thông tin mô hình tại [đây](https://huggingface.co/capleaf/viXTTS)

Bạn có thể dùng demo này với mục đích:
- ✅Mục đích cá nhân, học tập, nghiên cứu, thử nghiệm

Bạn **KHÔNG** được dùng demo này với mục đích:
- ❌Mục đích trái đạo đức, vi phạm pháp luật Việt Nam
- ❌Tạo ra nội dung gây thù ghét, kỳ thị, bạo lực hoặc nội dung vi phạm bản quyền
- ❌Giả mạo danh tính hoặc gây hiểu nhầm rằng nội dung được tạo ra bởi một cá nhân hoặc tổ chức khác

In [2]:
# @title 1. ⚙️ **Cài đặt**
# @markdown 👈Nhấn nút này để cài đặt (~5 phút)
# Change timezone to Vietnam
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Asia/Ho_Chi_Minh /etc/localtime
!date

print(" > Cài đặt thư viện...")
!rm -rf TTS/
!git clone --branch add-vietnamese-xtts -q https://github.com/thinhlpg/TTS.git
!pip install --use-deprecated=legacy-resolver -q -e TTS
!pip install deepspeed -q
!pip install -q vinorm==2.0.7
!pip install -q cutlet
!pip install -q unidic==1.1.0
!pip install -q underthesea
!pip install -q gradio==4.35
!pip install deepfilternet==0.5.6 -q

import os
from huggingface_hub import snapshot_download


os.system("python -m unidic download")
print(" > Tải mô hình...")
snapshot_download(repo_id="thinhlpg/viXTTS",
                  repo_type="model",
                  local_dir="model")

from IPython.display import clear_output
clear_output()
print(" > ✅ Cài đặt hoàn tất, bạn hãy chạy tiếp các bước tiếp theo nhé!")
quit()

 > ✅ Cài đặt hoàn tất, bạn hãy chạy tiếp các bước tiếp theo nhé!


In [4]:
# The inference code is adopted from https://github.com/coqui-ai/TTS/blob/dev/TTS/demos/xtts_ft_demo/xtts_demo.py
# @title 2. 🤗 **Sử dụng**
# @markdown 👈 Nhấn để chạy.
# @markdown Nếu gặp lỗi thì cũng nhấn nút này nhé!

# @markdown Lần đầu chạy sẽ hơi lâu, bạn chờ tí nhé!

# @markdown Kết quả sẽ được lưu vào `/content/output`

# @markdown Chọn ngôn ngữ:
language = "Tiếng Việt" # @param ["Tiếng Việt", "Tiếng Anh","Tiếng Tây Ban Nha", "Tiếng Pháp","Tiếng Đức","Tiếng Ý", "Tiếng Bồ Đào Nha", "Tiếng Ba Lan", "Tiếng Thổ Nhĩ Kỳ", "Tiếng Nga", "Tiếng Hà Lan", "Tiếng Séc", "Tiếng Ả Rập", "Tiếng Trung (giản thể)", "Tiếng Nhật", "Tiếng Hungary", "Tiếng Hàn", "Tiếng Hindi"]
# @markdown Văn bản để đọc. Độ dài tối thiểu mỗi câu nên từ 10 từ để đặt kết quả tốt nhất.
input_text ="\"Chào mừng bạn đến với hệ thống trợ lý ảo thông minh. Tôi có thể hỗ trợ bạn đọc văn bản, tìm kiếm thông tin, dịch thuật, hoặc thậm chí là hướng dẫn cách nấu ăn. Bạn chỉ cần nói yêu cầu của mình, và tôi sẽ cố gắng thực hiện chính xác và nhanh chóng. Mọi dữ liệu được xử lý với độ bảo mật cao để đảm bảo quyền riêng tư cho người dùng. Hãy thử một câu lệnh bất kỳ để bắt đầu, ví dụ như: 'Hôm nay thời tiết thế nào?' hoặc 'Đọc cho tôi một đoạn thơ.'\"" # @param {type:"string"}
# @markdown Chọn giọng mẫu:
reference_audio = "model/samples/audio.wav" # @param [ "model/user_sample.wav",  "model/vi_sample.wav",  "model/samples/nam-calm.wav",  "model/samples/nam-cham.wav",  "model/samples/nam-nhanh.wav",  "model/samples/nam-truyen-cam.wav",  "model/samples/nu-calm.wav",  "model/samples/nu-cham.wav",  "model/samples/nu-luu-loat.wav",  "model/samples/nu-nhan-nha.wav",  "model/samples/nu-nhe-nhang.wav","model/samples/audio.wav"]
# @markdown Tự động chuẩn hóa chữ (VD: 20/11 -> hai mươi tháng mười một)
normalize_text = True # @param {type:"boolean"}
# @markdown In chi tiết xử lý
verbose = True # @param {type:"boolean"}
# @markdown Lưu từng câu thành file riêng lẻ.
output_chunks = False # @param {type:"boolean"}

from IPython.display import clear_output
def cry_and_quit():
    clear_output()
    print("> Lỗi rồi huhu 😭😭, bạn hãy nhấn chạy lại phần này nhé!")
    quit()

import os
import string
import unicodedata
from datetime import datetime
from pprint import pprint

import torch
import torchaudio
from tqdm import tqdm
from underthesea import sent_tokenize
from unidecode import unidecode

try:
    from vinorm import TTSnorm
    from TTS.tts.configs.xtts_config import XttsConfig
    from TTS.tts.models.xtts import Xtts
except:
    cry_and_quit()

# Load model
def clear_gpu_cache():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()


def load_model(xtts_checkpoint, xtts_config, xtts_vocab):
    clear_gpu_cache()
    if not xtts_checkpoint or not xtts_config or not xtts_vocab:
        return "You need to run the previous steps or manually set the `XTTS checkpoint path`, `XTTS config path`, and `XTTS vocab path` fields !!"
    config = XttsConfig()
    config.load_json(xtts_config)
    XTTS_MODEL = Xtts.init_from_config(config)
    print("Loading XTTS model! ")
    XTTS_MODEL.load_checkpoint(config,
                               checkpoint_path=xtts_checkpoint,
                               vocab_path=xtts_vocab,
                               use_deepspeed=True)
    if torch.cuda.is_available():
        XTTS_MODEL.cuda()

    print("Model Loaded!")
    return XTTS_MODEL


def get_file_name(text, max_char=50):
    filename = text[:max_char]
    filename = filename.lower()
    filename = filename.replace(" ", "_")
    filename = filename.translate(str.maketrans("", "", string.punctuation.replace("_", "")))
    filename = unidecode(filename)
    current_datetime = datetime.now().strftime("%m%d%H%M%S")
    filename = f"{current_datetime}_{filename}"
    return filename


def calculate_keep_len(text, lang):
    if lang in ["ja", "zh-cn"]:
        return -1

    word_count = len(text.split())
    num_punct = (
        text.count(".")
        + text.count("!")
        + text.count("?")
        + text.count(",")
    )

    if word_count < 5:
        return 15000 * word_count + 2000 * num_punct
    elif word_count < 10:
        return 13000 * word_count + 2000 * num_punct
    return -1


def normalize_vietnamese_text(text):
    text = (
        TTSnorm(text, unknown=False, lower=False, rule=True)
        .replace("..", ".")
        .replace("!.", "!")
        .replace("?.", "?")
        .replace(" .", ".")
        .replace(" ,", ",")
        .replace('"', "")
        .replace("'", "")
        .replace("AI", "Ây Ai")
        .replace("A.I", "Ây Ai")
    )
    return text


def run_tts(XTTS_MODEL, lang, tts_text, speaker_audio_file,
            normalize_text= True,
            verbose=False,
            output_chunks=False):
    """
    Run text-to-speech (TTS) synthesis using the provided XTTS_MODEL.

    Args:
        XTTS_MODEL: A pre-trained TTS model.
        lang (str): The language of the input text.
        tts_text (str): The text to be synthesized into speech.
        speaker_audio_file (str): Path to the audio file of the speaker to condition the synthesis on.
        normalize_text (bool, optional): Whether to normalize the input text. Defaults to True.
        verbose (bool, optional): Whether to print verbose information. Defaults to False.
        output_chunks (bool, optional): Whether to save synthesized speech chunks separately. Defaults to False.

    Returns:
        str: Path to the synthesized audio file.
    """

    if XTTS_MODEL is None or not speaker_audio_file:
        return "You need to run the previous step to load the model !!", None, None

    output_dir = "./output"
    os.makedirs(output_dir, exist_ok=True)

    gpt_cond_latent, speaker_embedding = XTTS_MODEL.get_conditioning_latents(
        audio_path=speaker_audio_file,
        gpt_cond_len=XTTS_MODEL.config.gpt_cond_len,
        max_ref_length=XTTS_MODEL.config.max_ref_len,
        sound_norm_refs=XTTS_MODEL.config.sound_norm_refs,
    )

    if normalize_text and lang == "vi":
        # Bug on google colab
        try:
            tts_text = normalize_vietnamese_text(tts_text)
        except:
            cry_and_quit()

    if lang in ["ja", "zh-cn"]:
        tts_texts = tts_text.split("。")
    else:
        tts_texts = sent_tokenize(tts_text)

    if verbose:
        print("Text for TTS:")
        pprint(tts_texts)

    wav_chunks = []
    for text in tqdm(tts_texts):
        if text.strip() == "":
            continue

        wav_chunk = XTTS_MODEL.inference(
            text=text,
            language=lang,
            gpt_cond_latent=gpt_cond_latent,
            speaker_embedding=speaker_embedding,
            temperature=0.3,
            length_penalty=1.0,
            repetition_penalty=10.0,
            top_k=30,
            top_p=0.85,
        )

        # Quick hack for short sentences
        keep_len = calculate_keep_len(text, lang)
        wav_chunk["wav"] = torch.tensor(wav_chunk["wav"][:keep_len])

        if output_chunks:
            out_path = os.path.join(output_dir, f"{get_file_name(text)}.wav")
            torchaudio.save(out_path, wav_chunk["wav"].unsqueeze(0), 24000)
            if verbose:
                print(f"Saved chunk to {out_path}")

        wav_chunks.append(wav_chunk["wav"])

    out_wav = torch.cat(wav_chunks, dim=0).unsqueeze(0)
    out_path = os.path.join(output_dir, f"{get_file_name(tts_text)}.wav")
    torchaudio.save(out_path, out_wav, 24000)

    if verbose:
        print(f"Saved final file to {out_path}")

    return out_path


language_code_map = {
    "Tiếng Việt": "vi",
    "Tiếng Anh": "en",
    "Tiếng Tây Ban Nha": "es",
    "Tiếng Pháp": "fr",
    "Tiếng Đức": "de",
    "Tiếng Ý": "it",
    "Tiếng Bồ Đào Nha": "pt",
    "Tiếng Ba Lan": "pl",
    "Tiếng Thổ Nhĩ Kỳ": "tr",
    "Tiếng Nga": "ru",
    "Tiếng Hà Lan": "nl",
    "Tiếng Séc": "cs",
    "Tiếng Ả Rập": "ar",
    "Tiếng Trung (giản thể)": "zh-cn",
    "Tiếng Nhật": "ja",
    "Tiếng Hungary": "hu",
    "Tiếng Hàn": "ko",
    "Tiếng Hindi": "hi"
}

print("> Đang nạp mô hình...")
try:
    if not vixtts_model:
        vixtts_model = load_model(xtts_checkpoint="model/model.pth",
                                xtts_config="model/config.json",
                                xtts_vocab="model/vocab.json")
except:
    vixtts_model = load_model(xtts_checkpoint="model/model.pth",
                                xtts_config="model/config.json",
                                xtts_vocab="model/vocab.json")
clear_output()
print("> Đã nạp mô hình")

if not os.path.exists(reference_audio):
    print("⚠️⚠️⚠️Bạn chưa tải file âm thanh lên. Hãy chọn giọng khác, hoặc tải file của bạn lên ở bên dưới.⚠️⚠️⚠️")
    audio_file="/content/model/vi_sample.wav"
else:
    audio_file = run_tts(vixtts_model,
            lang=language_code_map[language],
            tts_text=input_text,
            speaker_audio_file=reference_audio,
            normalize_text=normalize_text,
            verbose=verbose,
            output_chunks=output_chunks,)

from IPython.display import Audio
Audio(audio_file)

> Đã nạp mô hình
⚠️⚠️⚠️Bạn chưa tải file âm thanh lên. Hãy chọn giọng khác, hoặc tải file của bạn lên ở bên dưới.⚠️⚠️⚠️


In [ ]:
# @title 🎤 **Tải file âm thanh của bạn lên**
# @markdown Để đạt chất lượng tốt nhất, hãy tham khảo file '/content/model/vi_sample.wav'
# @
import os
import locale
from google.colab import files

denoise = True # @param {type:"boolean"}

# Upload the audio file
uploaded = files.upload()
for filename in uploaded.keys():
    # Convert the audio file to WAV format using ffmpeg
    uploaded_dir = os.path.dirname(filename)
    if denoise:
        !deepFilter "{filename}"
        !ffmpeg -i "{filename.replace('.wav', '_DeepFilterNet3.wav')}" -ac 1 -ar 22050 -vn /content/model/user_sample.wav -y -hide_banner -loglevel error
        os.remove(filename.replace('.wav', '_DeepFilterNet3.wav'))
    else:
        !ffmpeg -i "{filename}" -ac 1 -ar 22050 -vn /content/model/user_sample.wav -y -hide_banner -loglevel error
        os.remove(filename)
    break

from IPython.display import Audio, clear_output
clear_output()
print("> Đã tải file âm thanh lên")
Audio("/content/model/user_sample.wav")

In [ ]:
# @title ⏬ **Lưu kết quả vào Drive**
# @markdown Chạy phần này để lưu kết quả vào Google Drive của bạn
# @markdown `/content/drive/MyDrive/vixtts-output`

from google.colab import drive
drive.mount('/content/drive')

# Save the output folder in "vixtts-output" in Google Drive, without overwriting existing files
!cp -n -r /content/output/* /content/drive/MyDrive/vixtts-output
print("> Đã lưu kết quả vào Google Drive")

In [ ]:
# @title ⚠️ **Dọn kết quả**
# @markdown Chạy phần này để xóa toàn bộ file trong `/content/output`
import shutil
shutil.rmtree('/content/output')
print("Đã xóa toàn bộ file trong /content/output")

In [3]:
# The inference code is adopted from https://github.com/coqui-ai/TTS/blob/dev/TTS/demos/xtts_ft_demo/xtts_demo.py
# @title 5. 🤗 **Sử dụng chạy PDF**
# @markdown 👈 Nhấn để chạy.
# @markdown Nếu gặp lỗi thì cũng nhấn nút này nhé!

# @markdown Lần đầu chạy sẽ hơi lâu, bạn chờ tí nhé!

# @markdown Kết quả sẽ được lưu vào `/content/output`

# @markdown Chọn ngôn ngữ:
language = "Tiếng Việt" # @param ["Tiếng Việt", "Tiếng Anh","Tiếng Tây Ban Nha", "Tiếng Pháp","Tiếng Đức","Tiếng Ý", "Tiếng Bồ Đào Nha", "Tiếng Ba Lan", "Tiếng Thổ Nhĩ Kỳ", "Tiếng Nga", "Tiếng Hà Lan", "Tiếng Séc", "Tiếng Ả Rập", "Tiếng Trung (giản thể)", "Tiếng Nhật", "Tiếng Hungary", "Tiếng Hàn", "Tiếng Hindi"]
# @markdown Văn bản để đọc. Độ dài tối thiểu mỗi câu nên từ 10 từ để đặt kết quả tốt nhất.
input_text ="Đoạn văn trong pdf sẽ hiển thị tại đây" # @param {type:"string"}
# @markdown Chọn giọng mẫu:
reference_audio = "audio.wav" # @param [ "model/user_sample.wav",  "model/vi_sample.wav",  "model/samples/nam-calm.wav",  "model/samples/nam-cham.wav",  "model/samples/nam-nhanh.wav",  "model/samples/nam-truyen-cam.wav",  "model/samples/nu-calm.wav",  "model/samples/nu-cham.wav",  "model/samples/nu-luu-loat.wav",  "model/samples/nu-nhan-nha.wav",  "model/samples/nu-nhe-nhang.wav","audio.wav"]
# @markdown Chọn file PDF:
reference_pdf = "media/thatlaccoinguoi.pdf" # @param ["media/thatlaccoinguoi.pdf"]
# @markdown Tự động chuẩn hóa chữ (VD: 20/11 -> hai mươi tháng mười một)
normalize_text = True # @param {type:"boolean"}
# @markdown In chi tiết xử lý
verbose = True # @param {type:"boolean"}
# @markdown Lưu từng câu thành file riêng lẻ.
output_chunks = True # @param {type:"boolean"}

from IPython.display import clear_output
def cry_and_quit():
    clear_output()
    print("> Lỗi rồi huhu 😭😭, bạn hãy nhấn chạy lại phần này nhé!")
    quit()

import os
import string
import unicodedata
from datetime import datetime
from pprint import pprint

import torch
import torchaudio
from tqdm import tqdm
from underthesea import sent_tokenize
from unidecode import unidecode
import pdfplumber

try:
    from vinorm import TTSnorm
    from TTS.tts.configs.xtts_config import XttsConfig
    from TTS.tts.models.xtts import Xtts
except:
    cry_and_quit()

# Load model
def clear_gpu_cache():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()


def load_model(xtts_checkpoint, xtts_config, xtts_vocab):
    clear_gpu_cache()
    if not xtts_checkpoint or not xtts_config or not xtts_vocab:
        return "You need to run the previous steps or manually set the `XTTS checkpoint path`, `XTTS config path`, and `XTTS vocab path` fields !!"
    config = XttsConfig()
    config.load_json(xtts_config)
    XTTS_MODEL = Xtts.init_from_config(config)
    print("Loading XTTS model! ")
    XTTS_MODEL.load_checkpoint(config,
                               checkpoint_path=xtts_checkpoint,
                               vocab_path=xtts_vocab,
                               use_deepspeed=True)
    if torch.cuda.is_available():
        XTTS_MODEL.cuda()

    print("Model Loaded!")
    return XTTS_MODEL


def get_file_name(text, max_char=50):
    filename = text[:max_char]
    filename = filename.lower()
    filename = filename.replace(" ", "_")
    filename = filename.translate(str.maketrans("", "", string.punctuation.replace("_", "")))
    filename = unidecode(filename)
    current_datetime = datetime.now().strftime("%m%d%H%M%S")
    filename = f"{current_datetime}_{filename}"
    return filename


def calculate_keep_len(text, lang):
    if lang in ["ja", "zh-cn"]:
        return -1

    word_count = len(text.split())
    num_punct = (
        text.count(".")
        + text.count("!")
        + text.count("?")
        + text.count(",")
    )

    if word_count < 5:
        return 15000 * word_count + 2000 * num_punct
    elif word_count < 10:
        return 13000 * word_count + 2000 * num_punct
    return -1


def normalize_vietnamese_text(text):
    text = (
        TTSnorm(text, unknown=False, lower=False, rule=True)
        .replace("..", ".")
        .replace("!.", "!")
        .replace("?.", "?")
        .replace(" .", ".")
        .replace(" ,", ",")
        .replace('"', "")
        .replace("'", "")
        .replace("AI", "Ây Ai")
        .replace("A.I", "Ây Ai")
    )
    return text


def run_tts(XTTS_MODEL, lang, tts_text, speaker_audio_file,
            normalize_text=True,
            verbose=False,
            output_chunks=False):
    """
    Chuyển văn bản thành giọng nói với mô hình XTTS. Chỉ lưu ra 1 file duy nhất.
    Hiển thị phần trăm tiến trình khi đang xử lý.
    """
    if XTTS_MODEL is None or not speaker_audio_file:
        return "Bạn cần tải mô hình và audio mẫu trước khi chạy!", None, None

    output_dir = "./output"
    os.makedirs(output_dir, exist_ok=True)

    # Lấy đặc trưng giọng nói từ file mẫu
    gpt_cond_latent, speaker_embedding = XTTS_MODEL.get_conditioning_latents(
        audio_path=speaker_audio_file,
        gpt_cond_len=XTTS_MODEL.config.gpt_cond_len,
        max_ref_length=XTTS_MODEL.config.max_ref_len,
        sound_norm_refs=XTTS_MODEL.config.sound_norm_refs,
    )

    if normalize_text and lang == "vi":
        try:
            tts_text = normalize_vietnamese_text(tts_text)
        except:
            cry_and_quit()

    # Tách văn bản thành từng câu
    if lang in ["ja", "zh-cn"]:
        tts_texts = tts_text.split("。")
    else:
        tts_texts = sent_tokenize(tts_text)

    if verbose:
        print("📜 Văn bản sẽ đọc:")
        pprint(tts_texts)

    # Tổng hợp từng câu và ghép lại
    wav_chunks = []
    total = len(tts_texts)
    for i, text in enumerate(tts_texts):
        if text.strip() == "":
            continue

        print(f"\r🎤 Đang chuyển đổi thành giọng nói... {int((i + 1) / total * 100)}%", end="", flush=True)

        wav_chunk = XTTS_MODEL.inference(
            text=text,
            language=lang,
            gpt_cond_latent=gpt_cond_latent,
            speaker_embedding=speaker_embedding,
            temperature=0.3,
            length_penalty=1.0,
            repetition_penalty=10.0,
            top_k=30,
            top_p=0.85,
        )

        # Cắt ngắn nếu câu quá ngắn
        keep_len = calculate_keep_len(text, lang)
        wav_chunk["wav"] = torch.tensor(wav_chunk["wav"][:keep_len])

        wav_chunks.append(wav_chunk["wav"])

    print("\n✅ Hoàn tất chuyển đổi giọng nói.")

    # Gộp tất cả lại thành một tệp .wav duy nhất
    out_wav = torch.cat(wav_chunks, dim=0).unsqueeze(0)
    out_path = os.path.join(output_dir, f"{get_file_name(tts_text)}.wav")
    torchaudio.save(out_path, out_wav, 24000)

    print(f"💾 Đã lưu tệp âm thanh vào: {out_path}")

    return out_path


language_code_map = {
    "Tiếng Việt": "vi",
    "Tiếng Anh": "en",
    "Tiếng Tây Ban Nha": "es",
    "Tiếng Pháp": "fr",
    "Tiếng Đức": "de",
    "Tiếng Ý": "it",
    "Tiếng Bồ Đào Nha": "pt",
    "Tiếng Ba Lan": "pl",
    "Tiếng Thổ Nhĩ Kỳ": "tr",
    "Tiếng Nga": "ru",
    "Tiếng Hà Lan": "nl",
    "Tiếng Séc": "cs",
    "Tiếng Ả Rập": "ar",
    "Tiếng Trung (giản thể)": "zh-cn",
    "Tiếng Nhật": "ja",
    "Tiếng Hungary": "hu",
    "Tiếng Hàn": "ko",
    "Tiếng Hindi": "hi"
}

print("> Đang nạp mô hình...")

# load file pdf
# def read_pdf_text(pdf_path):
#     text = ""
#     with pdfplumber.open(pdf_path) as pdf:
#         total_pages = len(pdf.pages)
#         for i, page in enumerate(pdf.pages):
#             page_text = page.extract_text()
#             if page_text:
#                 # Xử lý từng dòng
#                 lines = [line.strip() for line in page_text.split('\n') if line.strip()]
#                 cleaned = ""
#                 for j, line in enumerate(lines):
#                     cleaned += line
#                     # Nếu dòng không kết thúc bằng dấu câu và chưa phải dòng cuối -> nối dòng
#                     if not line.endswith(('.', '!', '?')) and j < len(lines) - 1:
#                         cleaned += " "
#                     else:
#                         cleaned += "\n"
#                 text += cleaned + "\n"
#             # Hiển thị tiến độ %
#             percent = int((i + 1) / total_pages * 100)
#             print(f"\r📖 Đang đọc PDF... {percent}%", end="", flush=True)
#     print("\n✅ Đã đọc xong PDF.")
#     return text

def read_pdf_text(pdf_path, max_chars=1000):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        total_pages = len(pdf.pages)
        for i, page in enumerate(pdf.pages):
            page_text = page.extract_text()
            if page_text:
                # Xử lý từng dòng
                lines = [line.strip() for line in page_text.split('\n') if line.strip()]
                cleaned = ""
                for j, line in enumerate(lines):
                    cleaned += line
                    if not line.endswith(('.', '!', '?')) and j < len(lines) - 1:
                        cleaned += " "
                    else:
                        cleaned += "\n"
                text += cleaned + "\n"
                # Nếu đã đủ số ký tự yêu cầu thì dừng đọc
                if len(text) >= max_chars:
                    text = text[:max_chars]
                    break
            percent = int((i + 1) / total_pages * 100)
            print(f"\r📖 Đang đọc PDF... {percent}%", end="", flush=True)
    print("\n✅ Đã đọc xong PDF.")
    return text

input_text = read_pdf_text(reference_pdf)
if not input_text.strip():
    print("❌ Không tìm thấy nội dung trong PDF.")
    raise SystemExit

try:
    if not vixtts_model:
        vixtts_model = load_model(xtts_checkpoint="model/model.pth",
                                xtts_config="model/config.json",
                                xtts_vocab="model/vocab.json")
except:
    vixtts_model = load_model(xtts_checkpoint="model/model.pth",
                                xtts_config="model/config.json",
                                xtts_vocab="model/vocab.json")
clear_output()
print("> Đã nạp mô hình")

if not os.path.exists(reference_audio):
    print("⚠️⚠️⚠️Bạn chưa tải file âm thanh lên. Hãy chọn giọng khác, hoặc tải file của bạn lên ở bên dưới.⚠️⚠️⚠️")
    audio_file="/content/model/vi_sample.wav"
if not os.path.exists(reference_pdf):
    print("⚠️⚠️⚠️Bạn chưa có file pdf, hoặc tải file của bạn lên ở bên dưới.⚠️⚠️⚠️")
    audio_file="/content/media/thatlaccoinguoi.pdf"
else:
    audio_file = run_tts(vixtts_model,
            lang=language_code_map[language],
            tts_text=input_text,
            speaker_audio_file=reference_audio,
            normalize_text=normalize_text,
            verbose=verbose,
            output_chunks=output_chunks,)

from IPython.display import Audio
Audio(audio_file)

> Đã nạp mô hình
⚠️⚠️⚠️Bạn chưa tải file âm thanh lên. Hãy chọn giọng khác, hoặc tải file của bạn lên ở bên dưới.⚠️⚠️⚠️


RuntimeError: Failed to open the input "audio.wav" (No such file or directory).
Exception raised from get_input_format_context at /__w/audio/audio/pytorch/audio/src/libtorio/ffmpeg/stream_reader/stream_reader.cpp:42 (most recent call first):
frame #0: c10::Error::Error(c10::SourceLocation, std::string) + 0x96 (0x7fefe036c1b6 in /usr/local/lib/python3.11/dist-packages/torch/lib/libc10.so)
frame #1: c10::detail::torchCheckFail(char const*, char const*, unsigned int, std::string const&) + 0x64 (0x7fefe0315a76 in /usr/local/lib/python3.11/dist-packages/torch/lib/libc10.so)
frame #2: <unknown function> + 0x42034 (0x7fefe00a8034 in /usr/local/lib/python3.11/dist-packages/torio/lib/libtorio_ffmpeg4.so)
frame #3: torio::io::StreamingMediaDecoder::StreamingMediaDecoder(std::string const&, std::optional<std::string> const&, std::optional<std::map<std::string, std::string, std::less<std::string>, std::allocator<std::pair<std::string const, std::string> > > > const&) + 0x14 (0x7fefe00aaa34 in /usr

In [ ]:
!pip install pdfplumber

In [ ]:
#Xoá thư mục
# @title 📴 **Xoá thư mục**
# @markdown lỡ bị nhiều file rác quá thì xoá!
import shutil
shutil.rmtree('/content/output1')
print("Đã xóa toàn bộ file trong /content/output1")

In [ ]:
# Chuyển đổi file
# @title 📴 **Chuyển đổi file m4r => wav**
# @markdown file âm demo không phải wav thì phải đổi!
from pydub import AudioSegment

# Đường dẫn đến file .m4a bạn đã upload
m4a_path = "/content/sample.m4a" # @param {type:"string"}
wav_path = "/content/sample.wav" # @param {type:"string"}

# Đọc file m4a và xuất thành wav
audio = AudioSegment.from_file(m4a_path, format="m4a")
audio.export(wav_path, format="wav")

print("✅ Đã chuyển sang WAV:", wav_path)

In [ ]:
# @title 📴 **Tắt Demo**
# @markdown Khi chạy xong thì bạn hãy tắt demo để tiết kiệm GPU nhé!
from google.colab import runtime
runtime.unassign()